In [ ]:
!pip install datasets

In [4]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer, pipeline
from datasets import load_dataset

In [7]:
import pandas as pd
from datasets import Dataset

# Read the CSV file
df = pd.read_csv("/content/transcript_clean.csv")

In [13]:
df

,text,label
0,"redness, swelling, and difficulty walking",Knee pain
1,I had an accident and my wound was open when I...,Open wound
2,My cut yellow or greenish-colored pus,Infected wound
3,I had a pain in my knee when I was swimming,Knee pain
4,"My body feels weak although I eat a lot, why?",Body feels weak
...,...,...
5323,There's swelling in my neck and it hurts.,Neck pain
5324,i was travelling by ship and i feel dizzy.,Feeling dizzy
5325,"I am always cold, even when I am wearing layers",Feeling cold
5326,My back hurts so much I can't bend down to tie...,Back pain


In [8]:
# Convert the DataFrame to a dictionary
data_dict = {"text": df["phrase"], "label": df["prompt"]}  # Modify based on your CSV structure

# Create a dataset object
bio_asq = Dataset.from_dict(data_dict)

# Split the dataset
bio_asq = bio_asq.train_test_split(test_size=0.2)


In [9]:
bio_asq

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 5328
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1332
    })
})

In [10]:
import pandas as pd
# Convert the dataset to a dictionary
data_dict = bio_asq["train"].to_dict()
# Create a DataFrame from the dictionary
df = pd.DataFrame.from_dict(data_dict)

In [16]:
import pandas as pd
from transformers import AutoTokenizer

# Load your dataset
# Assuming df contains your dataset with columns 'phrase' and 'prompt'

# Initialize the BioBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")

# Tokenize the text data
questions = [q.strip() for q in df["text"]]
context = [q.strip() for q in df["label"]]
inputs = tokenizer(
    questions,
    context,
    max_length=384,
    truncation="only_second",
    return_offsets_mapping=True,
    padding="max_length",
)

# Extract start and end positions (assuming you have a column 'answers' containing answer start and end positions)
# Adjust this part according to your specific task
offset_mapping = inputs.pop("offset_mapping")
start_positions = []
end_positions = []


# Add tokenized data and start/end positions to the DataFrame
data = {
    'input_ids': inputs['input_ids'],
    'attention_mask': inputs['attention_mask'],
}
df_encoded = pd.DataFrame(data)

# Save the encoded DataFrame to a CSV file
df_encoded.to_csv('encoding_train.csv', index=False)



In [17]:
train = Dataset.from_pandas(df)

In [28]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate==0.21.0

In [26]:
import accelerate

In [29]:
import accelerate

from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import DefaultDataCollator
from datasets import load_metric
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

model = AutoModelForQuestionAnswering.from_pretrained("dmis-lab/biobert-v1.1")

data_collator = DefaultDataCollator()

# Your existing code to load model, data collator, etc.

training_args = TrainingArguments(
    output_dir="qa_model",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    report_to=[],
    logging_dir="./logs",
)

def compute_metrics(p):
    predictions, labels = p.predictions, p.label_ids

    # Convert tuples to numpy arrays
    predictions = np.array(predictions)
    labels = np.array(labels)

    # Assuming your model outputs logits and you want to get predictions
    predictions = np.argmax(predictions, axis=2)
    labels = labels

    # Flatten the predictions and labels
    predictions = predictions.flatten()
    labels = labels.flatten()

    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')
    f1 = f1_score(labels, predictions, average='macro')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,2.477474,0.354167,0.260477,0.290837,0.247846
2,No log,1.808595,0.460833,0.370514,0.375185,0.338151
3,No log,1.715480,0.468333,0.391463,0.378682,0.349819


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall

TrainOutput(global_step=450, training_loss=2.574782443576389, metrics={'train_runtime': 305.3194, 'train_samples_per_second': 23.582, 'train_steps_per_second': 1.474, 'total_flos': 1411002486374400.0, 'train_loss': 2.574782443576389, 'epoch': 3.0})

In [ ]:
results = trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
results

{'eval_loss': 1.7154799699783325,
 'eval_accuracy': 0.4683333333333333,
 'eval_precision': 0.3914633003210766,
 'eval_recall': 0.3786818726449856,
 'eval_f1': 0.34981909693491176,
 'eval_runtime': 8.136,
 'eval_samples_per_second': 73.746,
 'eval_steps_per_second': 4.671,
 'epoch': 3.0}